In [1]:
import pyspark

In [2]:
df=sqlContext.sql("select * from dog_food_csv")

In [3]:
df.show()

+---+---+----+---+-------+
 A| B| C| D|Spoiled|
+---+---+----+---+-------+
 4| 2|12.0| 3| 1.0|
 5| 6|12.0| 7| 1.0|
 6| 2|13.0| 6| 1.0|
 4| 2|12.0| 1| 1.0|
 4| 2|12.0| 3| 1.0|
 10| 3|13.0| 9| 1.0|
 8| 5|14.0| 5| 1.0|
 5| 8|12.0| 8| 1.0|
 6| 5|12.0| 9| 1.0|
 3| 3|12.0| 1| 1.0|
 9| 8|11.0| 3| 1.0|
 1| 10|12.0| 3| 1.0|
 1| 5|13.0| 10| 1.0|
 2| 10|12.0| 6| 1.0|
 1| 10|11.0| 4| 1.0|
 5| 3|12.0| 2| 1.0|
 4| 9|11.0| 8| 1.0|
 5| 1|11.0| 1| 1.0|
 4| 9|12.0| 10| 1.0|
 5| 8|10.0| 9| 1.0|
+---+---+----+---+-------+
only showing top 20 rows

In [4]:
df.columns

Out[ 8 ]: ['A', 'B', 'C', 'D', 'Spoiled']

In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
InputVector=VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol='features')

In [7]:
d2=InputVector.transform(df)['Spoiled','features']

In [8]:
d2.show()

+-------+-------------------+
Spoiled| features|
+-------+-------------------+
 1.0| [4.0,2.0,12.0,3.0]|
 1.0| [5.0,6.0,12.0,7.0]|
 1.0| [6.0,2.0,13.0,6.0]|
 1.0| [4.0,2.0,12.0,1.0]|
 1.0| [4.0,2.0,12.0,3.0]|
 1.0|[10.0,3.0,13.0,9.0]|
 1.0| [8.0,5.0,14.0,5.0]|
 1.0| [5.0,8.0,12.0,8.0]|
 1.0| [6.0,5.0,12.0,9.0]|
 1.0| [3.0,3.0,12.0,1.0]|
 1.0| [9.0,8.0,11.0,3.0]|
 1.0|[1.0,10.0,12.0,3.0]|
 1.0|[1.0,5.0,13.0,10.0]|
 1.0|[2.0,10.0,12.0,6.0]|
 1.0|[1.0,10.0,11.0,4.0]|
 1.0| [5.0,3.0,12.0,2.0]|
 1.0| [4.0,9.0,11.0,8.0]|
 1.0| [5.0,1.0,11.0,1.0]|
 1.0|[4.0,9.0,12.0,10.0]|
 1.0| [5.0,8.0,10.0,9.0]|
+-------+-------------------+
only showing top 20 rows

In [9]:
# train,test=d2.randomSplit([0.7,0.3])

In [10]:
from pyspark.ml.classification import RandomForestClassifier
rf=RandomForestClassifier(labelCol="Spoiled", featuresCol="features", numTrees=20)

results=rf.fit(d2)

In [11]:
print(results.featureImportances)

(4,[0,1,2,3],[0.0210271879562,0.0192967313427,0.93868737469,0.0209887060112])

In [12]:
results.featuresCol

Out[ 30 ]: Param(parent='RandomForestClassifier_b721d336873d', name='featuresCol', doc='features column name')

In [13]:
results.featureImportances

Out[ 31 ]: SparseVector(4, {0: 0.021, 1: 0.0193, 2: 0.9387, 3: 0.021})

In [14]:
print('C has the maximum impact in the output as per randomForest')

C has the maximum impact in the output as per randomForest

In [15]:
results=rf.fit(train).transform(test)

In [16]:

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol='Spoiled')
evaluator.evaluate(results)

Out[ 39 ]: 0.9992877492877492

In [17]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="Spoiled", featuresCol="features")

results_dt=dt.fit(train).transform(test)

In [18]:
evaluator.evaluate(results_dt)

Out[ 46 ]: 0.9935897435897436

In [19]:
print('Feature Importance cannot be given through Decision Trees')

Feature Importance cannot be given through Decision Trees

In [20]:
from pyspark.ml.classification import GBTClassifier

gbt=GBTClassifier(labelCol="Spoiled", featuresCol="features", maxIter=10)


In [21]:
results_gbt=gbt.fit(train).transform(test)

In [22]:
evaluator.evaluate(results_gbt)

Out[ 50 ]: 0.9855175688509021

In [23]:
print('From All three Random forests give the highest accuracy')

From All three Random forests give the highest accuracy